# Implementing BB84 protocol with cirq

This notebook is based on [lecture notes from TU Delft university](https://ocw.tudelft.nl/wp-content/uploads/LN_Week6.pdf)

## What is Quantum Key Distribution?

### First, private key cryptography

Imagine you've written a secret note that you only want your best friend to read. To keep it secret, you decide to scramble the letters in a specific way, a way that only you and your friend understand. This method of scrambling is your _key_. Now, even if someone else gets the note, they won't understand what it says because it's all scrambled up. Your friend, who knows the key, can unscramble the note to read the original message. This is the basic idea behind the private key, or symmetric, cryptography.

Private key cryptography is a type of encryption where the same key is used to both encrypt and decrypt the message. You use the key (a specific set of instructions) to convert your message into a secret code, and the same key is used to convert the secret code back into the original message. Because the same key is used for both processes, it's really important to keep the key secret. If someone else gets the key, they can decode all of your messages.

The main challenge in private key cryptography is securely sharing the key with the person you want to communicate with. If you're not careful, someone else might intercept the key and decode your messages. This is known as the _key distribution problem_.

### Quantum Key Distribution to the rescue!

Quantum Key Distribution (QKD) offers a solution to this problem using the principles of quantum mechanics. In the quantum world, measuring a system can change its state. This is called the _observer effect._ If someone tries to intercept the key and measure these qubits, they will inevitably change their states. Your best friend will notice these changes when he receives the qubits, so they will know there was an eavesdropper. QKD solves the key distribution problem by enabling you and your friend to detect eavesdropping. If there's no eavesdropping, you can use the sequence of qubits that you've sent and your friend received to create a secret key. If there is eavesdropping, you discard the key and try again.

BB84 is one of the protocols realizing the idea of QKD.

## Our battle plan and idea

Here we will not dive deep into the mathematics (physics) behind every aspect of the protocol. I will provide you with references if you want to dive deeper. **Within the scope of this notebook we are focused on building the BB84 protocol using cirq while allowing our solution not to be perfect because we are learning.**

## Part 1: Noise-free BB84

For the sake of the document, the sender will be called Alice and the receiver will be called Bob.

To implement noise-free BB84 we will follow these steps:
 1. Alice will generate two random binary (consisting only of `0`s and `1`s) strings - `alice_a`, and `alice_b` - of the same length.
 2. Alice will use her strings to prepare a quantum state (sequence of qubits) that she will send to Bob.
 3. Bob will generate a random binary string `bob_b` that he will use to try to _guess_ and decode Alice's message. Then Bob tells Alice that he received and measured the state he received.
 4. Alice and Bob exchange their `_b` strings. They discard all bits that do not match (from Alice's original message and Bob's guess-decoded message). They are left with `alice_kept_a` and `bob_kept_a` strings, respectively.
 5. Alice picks a random set of indices of `alice_kept_a` - `test_indices` and sends them to Bob. Alice and Bob use `test_indices` to create binary strings `alice_test_a` and `bob_test_a`. **If any of the bits miss-match they restart the procedure**
 6. They now use remaining indices (without `test_indices`) - `final_indices` - and treat is as a seed for key creation via the process of privacy amplification (we will discuss it later, do not worry!)
 
Let's get to it!

### Step 0

Import cirq and let's declare cirq's simulator (it enables quantum simulation!).

In [13]:
import cirq
import numpy as np

simulator = cirq.Simulator()

### Step 1
Our first step is to generate two random binary strings (represented as arrays of `0` and `1`s) - `alice_a` and `alice_b`. In the [previous screencast](https://www.youtube.com/watch?v=kANpuN2Ax0A) we build a quantum random number generator. We will leverage this solution right now!

We need to of course decide on the length of our initial strings. We will use a length that is a multiple of 4, so for `n` the length will be `4n`. On "average" in step 4 half of the bits would mismatch leaving us with `2n`. In step 6, the `final_indices` will again be half-length, leaving us with `n`.

In [14]:
from quantum_lib.qrng import QuantumRandomNumberGenerator

qrng = QuantumRandomNumberGenerator(simulator)

n = 4
string_length = 4 * n

alice_a = qrng.generate_binary_array(string_length)
alice_b = qrng.generate_binary_array(string_length)
print(alice_a)
print(alice_b)

[1 1 0 0 1 1 0 1 0 1 0 1 0 1 1 0]
[0 1 1 1 0 1 1 0 1 0 0 0 0 0 0 1]


### Step 2
Now it is time to prepare a quantum state that Alice will send to Bob. In our simplified version we will use good old H-gate. The idea is simple if the `alice_b[i] == 1` we will apply H-gate to that particular qubit. For sciency folks - we are changing _the computational basis_.

To do that we will first need to create an array of qubits of `string_length`. Recall that in cirq all qubits are in a "0" ($| 0 \rangle$) state initially. In classical computing, we would use the _NOT_ gate. It's quantum analog is called _X-gate_. Summing up our tactics looks like this:
 1. if `alice_a[idx] == 1` then apply `X` gate.
 2. If `alice_b[idx] == 1` then apply `H` gate.

In [15]:
quantum_channel = cirq.Circuit()
channel_qubits = cirq.NamedQubit.range(string_length, prefix="q_")
for idx in range(string_length):
    if alice_a[idx] == 1:
        quantum_channel.append(cirq.X(channel_qubits[idx]))
    if alice_b[idx] == 1:
        quantum_channel.append(cirq.H(channel_qubits[idx]))
print(quantum_channel)

q_0: ────X───────

q_1: ────X───H───

q_2: ────H───────

q_3: ────H───────

q_4: ────X───────

q_5: ────X───H───

q_6: ────H───────

q_7: ────X───────

q_8: ────H───────

q_9: ────X───────

q_11: ───X───────

q_13: ───X───────

q_14: ───X───────

q_15: ───H───────


We can see that the qubits that did not need either the `X` or `H` gate are not showing up. Do not worry they _will_ show up when we will measure the state! We called the variable `quantum_channel` to underscore the fact that on one side we have Alice preparing the qubits (that is - state) and on the other hand, we will have Bob reading the state (that is - the qubits).

### Step 3
In this step, Bob will try to "guess-decode" the `alice_a` string. How? He will generate his own `bob_b` string and whenever `bob_b[idx] == 1` he will "revert" the `H` gate. How? `H` gate has this awesome property that if you apply it twice you get your original state!

But what happens if `alice_b[idx] != bob_b[idx]`? That means that we have applied the `H` gate exactly once. And if you recall from our [previous screencast](https://www.youtube.com/watch?v=kANpuN2Ax0A) qubit in that state will give exactly random `0`, `1` measurements - so we will randomly measure `0` and `1`.

In [16]:
bob_b = qrng.generate_binary_array(string_length)
for idx in range(string_length):
    if bob_b[idx] == 1:
        quantum_channel.append(cirq.H(channel_qubits[idx]))
quantum_channel.append(cirq.measure(channel_qubits, key="bob_measurement"))

In [17]:
print(quantum_channel)

q_0: ────X───H───────M('bob_measurement')───
                     │
q_1: ────X───H───────M──────────────────────
                     │
q_2: ────H───────────M──────────────────────
                     │
q_3: ────H───────────M──────────────────────
                     │
q_4: ────X───────────M──────────────────────
                     │
q_5: ────X───H───H───M──────────────────────
                     │
q_6: ────H───H───────M──────────────────────
                     │
q_7: ────X───────────M──────────────────────
                     │
q_8: ────H───────────M──────────────────────
                     │
q_9: ────X───H───────M──────────────────────
                     │
q_10: ───────────────M──────────────────────
                     │
q_11: ───X───H───────M──────────────────────
                     │
q_12: ───────────────M──────────────────────
                     │
q_13: ───X───H───────M──────────────────────
                     │
q_14: ───X───────────M──────────────────────
   

In [18]:
simulation_result = simulator.simulate(quantum_channel)
bob_guess_decode = simulation_result.measurements["bob_measurement"]
print("Bob:", "Hey Alice, I received and measured state you've sent me!")
print(bob_guess_decode)

Bob: Hey Alice, I received and measured state you've sent me!
[0 0 1 0 1 1 0 1 0 0 0 0 0 0 1 0]


### Step 4
Okay, so now Alice and Bob can exchange their `_b` strings (Bob remains unaware of Alice's `alice_a` string. Alice remains unaware of Bob's `bob_guess_decode` string). They will then compare the `_b` strings and each of them will create a substring from `alice_a`/`bob_guess_decode` using only indices that match.

The interesting part is that if `_b` indices match, the corresponding `alice_a`/`bob_guess_decode` elements should also match because we used an appropriate number of `H` gates for that particular circuit.

Let's print all the strings to see what exactly happens

In [19]:
b_matching_indices = alice_b == bob_b
alice_kept_a = alice_a[b_matching_indices]
bob_kept_a = bob_guess_decode[b_matching_indices]
print("Alice b string:", alice_b)
print("Bob b string:  ", bob_b, "\n")
print("Alice a string:    ", alice_a)
print("Bob decoded string:", bob_guess_decode, "\n")
print("Alice kept:", alice_kept_a)
print("Bob kept:  ", bob_kept_a)

Alice b string: [0 1 1 1 0 1 1 0 1 0 0 0 0 0 0 1]
Bob b string:   [1 0 0 0 0 1 1 0 0 1 0 1 0 1 0 1] 

Alice a string:     [1 1 0 0 1 1 0 1 0 1 0 1 0 1 1 0]
Bob decoded string: [0 0 1 0 1 1 0 1 0 0 0 0 0 0 1 0] 

Alice kept: [1 1 0 1 0 0 1 0]
Bob kept:   [1 1 0 1 0 0 1 0]


## Part 2: Introducing Eve!
Okay, so now Alice and Bob have shared a string that matches precisely. They can use a process of privacy amplification (more on that later!) to create a secure key. Right now, our circuit does not have a concept of an _adversary_ (someone who would like to either change the message or learn the `alice_a` message). So let us focus next on that!

Eavesdropping Eve has multiple ways of attacking. In case of BB84 protocol we assume that Eve _cannot_ impersonate either Alice or Bob when they are communicating over classical channel - when they are sending classical bits (as opposed to sending qubits). But what Eve can _try to do_ is to eavesdrop. But as opposed to classical scenario **copying arbitrary qubits without introducing noise** is impossible. This is called the [no-cloning theorem](https://en.wikipedia.org/wiki/No-cloning_theorem). Basically, the very laws of nature make it impossible to copy the state of the qubit _without_ measuring it. But, if you recall - when we apply `H`-gate we have 50-50 chance of observing either `1` or `0`. So thinking like a bad actor we come up with a following strategy:
 1. We will intercept quantum message that goes from Alice to Bob.
 2. We will create Eve's `eve_b` to measure received state (we know the idea of the protocol, so we leverage that as an attacker)
 3. Eve then encodes the message again and send it to Bob
Eve needs to send the quantum state to Bob, because otherwise Bob will not be able to execute Step 3 and thus both Alice and Bob will think that either they simply lost the message or somebody intercepted it.

So now we will _recreate_ the circuit with Eve's actions.

In [20]:
# Alice prepares quantum state
alice_quantum_channel = cirq.Circuit()
alice_qubits = cirq.NamedQubit.range(string_length, prefix="aq_")
for idx in range(string_length):
    if alice_a[idx] == 1:
        alice_quantum_channel.append(cirq.X(alice_qubits[idx]))
    if alice_b[idx] == 1:
        alice_quantum_channel.append(cirq.H(alice_qubits[idx]))

# But instead of sending it to Bob directly, Eve intercepts, measures and send it further to Bob
eve_b  = qrng.generate_binary_array(string_length)
for idx in range(string_length):
    if eve_b[idx] == 1:
        alice_quantum_channel.append(cirq.H(alice_qubits[idx]))
alice_quantum_channel.append(cirq.measure(alice_qubits, key="eve_measurement"))
eve_simulation_result = simulator.simulate(alice_quantum_channel)
eve_guess_decode = eve_simulation_result.measurements["eve_measurement"]

# Now Eve will prepare new quantum state and send it to Bob.
eve_quantum_channel = cirq.Circuit()
eve_qubits = cirq.NamedQubit.range(string_length, prefix="eq_")
for idx in range(string_length):
    if eve_guess_decode[idx] == 1:
        eve_quantum_channel.append(cirq.X(eve_qubits[idx]))
    if eve_b[idx] == 1:
        eve_quantum_channel.append(cirq.H(eve_qubits[idx]))

# Now Bob takes the measurements
for idx in range(string_length):
    if bob_b[idx] == 1:
        eve_quantum_channel.append(cirq.H(eve_qubits[idx]))
eve_quantum_channel.append(cirq.measure(eve_qubits, key="bob_measurement"))
bob_simulation_result = simulator.simulate(eve_quantum_channel)
bob_guess_decode = bob_simulation_result.measurements["bob_measurement"]
print("Bob:", "Hey Alice, I received and measured state you've sent me!")

Bob: Hey Alice, I received and measured state you've sent me!


Now that was a lot! So lets see what results we observed.

In [21]:
print("Alice b string:", alice_b)
print("Eve b string:  ", eve_b)
print("Bob b string:  ", bob_b, "\n")

print("Alice a string:    ", alice_a)
print("Eve decoded string:", eve_guess_decode)
print("Bob decoded string:", bob_guess_decode, "\n")


Alice b string: [0 1 1 1 0 1 1 0 1 0 0 0 0 0 0 1]
Eve b string:   [1 0 1 1 0 1 0 1 0 1 1 1 1 0 0 0]
Bob b string:   [1 0 0 0 0 1 1 0 0 1 0 1 0 1 0 1] 

Alice a string:     [1 1 0 0 1 1 0 1 0 1 0 1 0 1 1 0]
Eve decoded string: [1 0 0 0 1 1 1 1 1 1 1 1 0 1 1 1]
Bob decoded string: [1 0 1 1 1 1 1 1 1 1 0 1 0 0 1 0] 



As we can see Eve created quite a mess! And learned _a lot_ about Alice's `alice_a` string. We will see how the protocol protects Alice and Bob from such interferences. Let us execute Step 4 again!

In [22]:
b_matching_indices = alice_b == bob_b
alice_kept_a = alice_a[b_matching_indices]
bob_kept_a = bob_guess_decode[b_matching_indices]
print("Alice b string:", alice_b)
print("Bob b string:  ", bob_b, "\n")
print("Alice a string:    ", alice_a)
print("Bob decoded string:", bob_guess_decode, "\n")
print("Alice kept:", alice_kept_a)
print("Bob kept:  ", bob_kept_a)

Alice b string: [0 1 1 1 0 1 1 0 1 0 0 0 0 0 0 1]
Bob b string:   [1 0 0 0 0 1 1 0 0 1 0 1 0 1 0 1] 

Alice a string:     [1 1 0 0 1 1 0 1 0 1 0 1 0 1 1 0]
Bob decoded string: [1 0 1 1 1 1 1 1 1 1 0 1 0 0 1 0] 

Alice kept: [1 1 0 1 0 0 1 0]
Bob kept:   [1 1 1 1 0 0 1 0]


### Step 5 (updated)

As you can clearly see _because_ of Eve's interference (_noise_) the `kept` strings already differ! Let's recall next step:
>  5. Alice picks a random set of indices of `alice_kept_a` - `test_indices` and sends them to Bob. Alice and Bob use `test_indices` to create binary strings `alice_test_a` and `bob_test_a`. **If any of the bits miss-match they restart the procedure**

Well, we might have restart the procedure quite many times. It turns out that our "noise-free" version does not handle noise really well. We will modify it slightly:
Steps 1-4 will remain the same. Then the fifth steps will become:
 5. Alice picks a random set of indices of `alice_kept_a` - `test_indices` and sends them to Bob. Alice and Bob use `test_indices` to create binary strings `alice_test_a` and `bob_test_a`. They agree on threshold `t`. If more than `t` bits missmatch than they restart protocol, otherwise they continue.

To build test indices we will create a binary array and then simply find indices of `1`s.

In [49]:
test_indices_source = qrng.generate_binary_array(len(alice_kept_a))
test_indices = np.where(test_indices_source == 1)[0]
print(f"Hey Bob, here are the test indices: {test_indices}")
alice_test_a = alice_kept_a[test_indices]
bob_test_a = bob_kept_a[test_indices]


Hey Bob, here are the test indices: [0 3 4 6]


In [50]:
threshold = 1
not_matching_elements = len(np.where(alice_test_a != bob_test_a))
if not_matching_elements > threshold:
    print("Restart the protocol")
else:
    print("Continue!")

Continue!


### Step 6 (Updated)

Originally, we were meant to perform privacy amplification. But due to Eve's interference Alice and Bob cannot be sure that even the remaining qubits measurements are the same. Do fix the potential errors we will use proces aptly named **Error correction**. That topic is _big_, _deep_ and _important_ in quantum computing, Here we will only provide high level overview and leverage relatively simple technique. You can ready on that in the [TU Delft's lecture notes](https://ocw.tudelft.nl/wp-content/uploads/LN_Week5.pdf).

Error correction, when performed over authenticated (that is - you are _sure_ who is a sender and a receiver) classical channel is called **Information reconciliation**. We will explore one technique to do that called **Syndrome coding**.

#### So what is information reconciliation?

Information reconciliation ensures the sender and the receiver,THank yo Alice and Bob, hold the same string of information. This is necessary because as we saw we are dealing with noise during the communication. The process (on a high level) works like this: Alice and Bob exchange error-correcting information to correct any discrepancies in their strings. The goal is to make sure that after reconciliation, the strings held by Alice and Bob are mostly-correct.

However, it's important to note that all classical communication between Alice and Bob is public, which means an eavesdropper (Eve) can also gain information from the error correction information they send across. Therefore, the process must be designed to minimize the information leakage to Eve.

Fun, right?

#### Syndrome coding using linear codes

In syndrome coding, the idea is to add extra bits to the original data being transmitted. These extra bits do not carry additional information per se, but they are there to help detect and correct errors that may occur during data transmission. Linear codes are one of the concrete implementations of syndrome coding.

First let's keep in mind what we are trying to do - **We want to detect and correct errors that might occur when information is transmitted across a noisy channel**.

Please note that At this moment we can actually make some assumptions about how many mismatching bits we need to detect and correct. In this tutorial we are not particularly rigorous (but the important point is that we can be rigorous and have strong guarantees). We will assume that _at most one bit_ can be off.

So what makes code linear? Let's say you have a list of binary strings that you want use as a codes. They will be called linear if when you take any two codes and add them together (modulo 2 or XOR) resulting code will also be in the original list. For math-savvy folks - the codes need to form a linear space. This property will be leveraged in a moment.

One of the most popular linear codes is called [Hamming(7, 4)](https://en.wikipedia.org/wiki/Hamming(7,4)). The coding will allow us to always correct single bit (if it is off).

First, we will describe procedure on a high level, then we will actually fill in missing details.

1. We used `test_indices` to obtain test strings. We will use remaining indices for further processing. The strings obtained from using those indices will be called `alice_final_a`, `bob_final_a`.
2. Alice will encode the message to obtain `encoded_final_a`. Alice will then send this message to Bob.
3. We know that `bob_final_a` is basically a sum of `alice_final_a` and some noise - which could be represented as binary vector of the same length and `alice_final_a`. Because we are using linear codes the following property holds `encode(a + b) == encode(a) + encode(b)` (I told you that this is important property!).
4. So Bob has message `bob_final_a == alice_final_a + noise`, using the linear property of linear code we have `encode(bob_final_a) == encode(alice_final_a + noise) == encode(alice_final_a) + encode(noise)`. If alice now will send him `encoded_final_a == encode(alice_final_a)`, then bob can subtract it from his encoded string to obtain encoded noise. Then he can `decode` his noise and subtract it his `decoded_noise` from `bob_final_a` to obtain `alice_final_a`.

Now that is a lot! So let's go step by step.

##### Step 6.1
That part is relatively simple - we will leverage `test_indices_source` defined before to create `final` strings.

In [53]:
final_indices = np.where(test_indices_source == 0)[0]
alice_final_a = alice_kept_a[final_indices]
bob_final_a = bob_kept_a[final_indices]
